In [ ]:
#Libraries
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import csv
from googlesearch import search
import spacy

In [ ]:
def obtener_titulos_peliculas():
    url = "https://trends.google.com/trends/trendingsearches/daily?geo=MX&hl=es"

    # Configurar Selenium
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(options=options)

    # Obtener la página con Selenium
    driver.get(url)

    try:
        # Esperar a que la página se cargue completamente (puedes ajustar el tiempo si es necesario)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '[ng-repeat="titlePart in titleArray"]'))
        )

        # Crear una lista para almacenar los datos
        datos = []

        # Intentar hacer clic en el botón "Cargar más" varias veces
        for _ in range(1):  # Puedes ajustar la cantidad de clics según sea necesario
            cargar_mas_button = driver.find_element(By.CLASS_NAME, 'feed-load-more-button')
            if cargar_mas_button.is_displayed():
                driver.execute_script("arguments[0].click();", cargar_mas_button)
                time.sleep(5)  # Esperar a que carguen los nuevos resultados
            else:
                break  # Si el botón no está visible, salimos del bucle

        # Obtener el HTML después de cargar completamente
        page_source = driver.page_source

        # Analizar el HTML de la página
        soup = BeautifulSoup(page_source, 'html.parser')

        # Encontrar los títulos de las películas
        titulos = soup.select('[ng-repeat="titlePart in titleArray"]')

        for indice, titulo in enumerate(titulos, start=1):
            titulo_texto = titulo.text.strip()
            #print(f"{indice}. {titulo_texto}")

            # Agregar los datos a la lista
            datos.append([titulo_texto])

        # Guardar los datos en un archivo CSV
        with open('resultados.csv', mode='w', newline='', encoding='utf-8') as archivo_csv:
            escritor_csv = csv.writer(archivo_csv)
            escritor_csv.writerow(['Título'])  # Escribir encabezado
            escritor_csv.writerows(datos)

    finally:
        # Cerrar el navegador
        driver.quit()

if __name__ == "__main__":
    obtener_titulos_peliculas()

In [7]:
# Lee el archivo csv generado arriba, ingresa a los link y corrobora palabras claves que le pasamos
def buscar_palabras_clave_en_pagina(enlace, palabras_clave):
    """
    Función para buscar palabras clave en el contenido de una página web.

    :param enlace: El enlace de la página a analizar.
    :param palabras_clave: Lista de palabras clave a buscar.
    :return: True si al menos una palabra clave se encuentra en la página, False de lo contrario.
    """
    # Configurar Selenium
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(options=options)

    try:
        # Obtener la página con Selenium
        driver.get(enlace)

        # Esperar a que la página se cargue completamente (puedes ajustar el tiempo si es necesario)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, 'body'))
        )

        # Obtener el contenido de la página
        page_source = driver.page_source

        # Analizar el HTML de la página
        soup = BeautifulSoup(page_source, 'html.parser')

        # Obtener el texto del contenido de la página
        contenido_pagina = soup.get_text().lower()

        # Buscar palabras clave en el contenido de la página
        for palabra_clave in palabras_clave:
            if palabra_clave.lower() in contenido_pagina:
                return True

        return False

    finally:
        # Cerrar el navegador
        driver.quit()

def obtener_titulos_desde_csv(csv_path, limite=5):
    # Configurar Selenium
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(options=options)

    try:
        # Crear una lista para almacenar los datos
        datos = []

        with open(csv_path, mode='r', encoding='utf-8') as archivo_csv:
            lector_csv = csv.reader(archivo_csv)
            next(lector_csv)  # Saltar la primera fila (encabezado)

            for indice, fila in enumerate(lector_csv, start=1):
                enlace = fila[1]  # La columna 1 contiene los enlaces
                titulo = fila[0]  # La columna 0 contiene los títulos

                # Verificar si el contenido de la página contiene palabras clave
                if buscar_palabras_clave_en_pagina(enlace, palabras_clave):
                    # Agregar los datos a la lista
                    datos.append([titulo, enlace])

                # Detener el procesamiento después de las primeras 5 filas
                if indice >= limite:
                    break

        # Imprimir los resultados
        for resultado in datos:
            print("Título:", resultado[0])
            print("Enlace:", resultado[1])
            print("-" * 50)

    finally:
        # Cerrar el navegador
        driver.quit()

if __name__ == "__main__":
    palabras_clave = ["Messi"]  # Personaliza las palabras clave según tus necesidades
    csv_path = 'resultados.csv'  # Ruta del archivo CSV
    limite_filas = 5  # Número de filas a procesar si queremos que chequee todo el csv le sacamos el limite

    obtener_titulos_desde_csv(csv_path, limite=limite_filas)


Exception ignored in: <function Service.__del__ at 0x0000015A843CCF40>
Traceback (most recent call last):
  File "c:\Users\ramir\AppData\Local\Programs\Python\Python312\Lib\site-packages\selenium\webdriver\common\service.py", line 185, in __del__
    self.stop()
  File "c:\Users\ramir\AppData\Local\Programs\Python\Python312\Lib\site-packages\selenium\webdriver\common\service.py", line 146, in stop
    self.send_remote_shutdown_command()
  File "c:\Users\ramir\AppData\Local\Programs\Python\Python312\Lib\site-packages\selenium\webdriver\common\service.py", line 126, in send_remote_shutdown_command
    request.urlopen(f"{self.service_url}/shutdown")
  File "c:\Users\ramir\AppData\Local\Programs\Python\Python312\Lib\urllib\request.py", line 215, in urlopen
    return opener.open(url, data, timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ramir\AppData\Local\Programs\Python\Python312\Lib\urllib\request.py", line 515, in open
    response = self._open(req, data)
         


TITULO NOTICIAS Y ENLACES (PRUEBAS)

In [4]:
def obtener_titulos_noticias():
    url = "https://trends.google.com/trends/trendingsearches/daily?geo=MX&hl=es"

    # Configurar Selenium
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(options=options)

    # Obtener la página con Selenium
    driver.get(url)

    try:
        # Esperar a que la página se cargue completamente (puedes ajustar el tiempo si es necesario)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '[ng-repeat="titlePart in titleArray"]'))
        )

        # Crear una lista para almacenar los datos
        datos = []

        # Intentar hacer clic en el botón "Cargar más" varias veces
        for _ in range(1):  # Puedes ajustar la cantidad de clics según sea necesario
            cargar_mas_button = driver.find_element(By.CLASS_NAME, 'feed-load-more-button')
            if cargar_mas_button.is_displayed():
                driver.execute_script("arguments[0].click();", cargar_mas_button)
                time.sleep(5)  # Esperar a que carguen los nuevos resultados
            else:
                break  # Si el botón no está visible, salimos del bucle

        # Obtener el HTML después de cargar completamente
        page_source = driver.page_source

        # Analizar el HTML de la página
        soup = BeautifulSoup(page_source, 'html.parser')

        # Encontrar los títulos y enlaces de las noticias
        noticias = soup.select('.summary-text a')

        for indice, noticia in enumerate(noticias, start=1):
            titulo_texto = noticia.find_parent('div', class_='details').find('div', class_='title').text.strip()
            enlace = noticia['href']  # Obtener el enlace directamente del atributo href
            # Agregar los datos a la lista
            datos.append([titulo_texto, enlace])

        # Guardar los datos en un archivo CSV
        with open('resultados.csv', mode='w', newline='', encoding='utf-8') as archivo_csv:
            escritor_csv = csv.writer(archivo_csv)
            escritor_csv.writerow(['Título', 'Enlace'])  # Escribir encabezado
            escritor_csv.writerows(datos)

    finally:
        # Cerrar el navegador
        driver.quit()

if __name__ == "__main__":
    obtener_titulos_noticias()


In [ ]:
def buscar_palabras_clave_en_pagina(enlace, palabras_clave):
    """
    Función para buscar palabras clave en el contenido de una página web.

    :param enlace: El enlace de la página a analizar.
    :param palabras_clave: Lista de palabras clave a buscar.
    :return: True si al menos una palabra clave se encuentra en la página, False de lo contrario.
    """
    # Configurar Selenium
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(options=options)

    try:
        # Obtener la página con Selenium
        driver.get(enlace)

        # Esperar a que la página se cargue completamente (puedes ajustar el tiempo si es necesario)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, 'body'))
        )

        # Obtener el contenido de la página
        page_source = driver.page_source

        # Analizar el HTML de la página
        soup = BeautifulSoup(page_source, 'html.parser')

        # Buscar palabras clave en el contenido de la página
        for palabra_clave in palabras_clave:
            if palabra_clave.lower() in soup.get_text().lower():
                return True

        return False

    finally:
        # Cerrar el navegador
        driver.quit()

def obtener_titulos_noticias():
    url = "https://trends.google.com/trends/trendingsearches/daily?geo=MX&hl=es"
    palabras_clave = ["México", "fútbol"]  # Personaliza las palabras clave según tus necesidades

    # Configurar Selenium
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(options=options)

    try:
        # Obtener la página con Selenium
        driver.get(url)

        # Esperar a que la página se cargue completamente (puedes ajustar el tiempo si es necesario)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '[ng-repeat="titlePart in titleArray"]'))
        )

        # Crear una lista para almacenar los datos
        datos = []

        # Contador para limitar el número de iteraciones
        iteraciones = 0

         # Intentar hacer clic en el botón "Cargar más" varias veces
        for _ in range(1):  # Puedes ajustar la cantidad de clics según sea necesario
            cargar_mas_button = driver.find_element(By.CLASS_NAME, 'feed-load-more-button')
            if cargar_mas_button.is_displayed():
                driver.execute_script("arguments[0].click();", cargar_mas_button)
                time.sleep(5)  # Esperar a que carguen los nuevos resultados
            else:
                break  # Si el botón no está visible, salimos del bucle
            
            # Obtener el HTML después de cargar completamente
            page_source = driver.page_source

            # Analizar el HTML de la página
            soup = BeautifulSoup(page_source, 'html.parser')

            # Encontrar los títulos y enlaces de las noticias
            noticias = soup.select('.summary-text a')

            for noticia in noticias:
                titulo_texto = noticia.find_parent('div', class_='details').find('div', class_='title').text.strip()
                enlace = noticia['href']  # Obtener el enlace directamente del atributo href

                # Verificar si el enlace contiene palabras clave y si la noticia ya está en la lista
                if buscar_palabras_clave_en_pagina(enlace, palabras_clave) and [titulo_texto, enlace] not in datos:
                    # Agregar los datos a la lista
                    datos.append([titulo_texto, enlace])

            # Incrementar el contador de iteraciones
            iteraciones += 1

            # Limitar el número de iteraciones (ajusta según sea necesario)
            if iteraciones >= 2:
                break

        # Imprimir los resultados
        for resultado in datos:
            print("Título:", resultado[0])
            print("Enlace:", resultado[1])
            print("-" * 50)

    finally:
        # Cerrar el navegador
        driver.quit()

if __name__ == "__main__":
    obtener_titulos_noticias()


In [6]:
def buscar_palabras_clave_en_pagina(enlace, palabras_clave):
    """
    Función para buscar palabras clave en el contenido de una página web.

    :param enlace: El enlace de la página a analizar.
    :param palabras_clave: Lista de palabras clave a buscar.
    :return: True si al menos una palabra clave se encuentra en la página, False de lo contrario.
    """
    # Configurar Selenium
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(options=options)

    try:
        # Obtener la página con Selenium
        driver.get(enlace)

        # Esperar a que la página se cargue completamente (puedes ajustar el tiempo si es necesario)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, 'body'))
        )

        # Obtener el contenido de la página
        page_source = driver.page_source

        # Analizar el HTML de la página
        soup = BeautifulSoup(page_source, 'html.parser')

        # Obtener el texto del contenido de la página
        contenido_pagina = soup.get_text().lower()

        # Buscar palabras clave en el contenido de la página
        for palabra_clave in palabras_clave:
            if palabra_clave.lower() in contenido_pagina:
                return True

        return False

    finally:
        # Cerrar el navegador
        driver.quit()

def obtener_titulos_desde_csv(csv_path, limite=5):
    # Configurar Selenium
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(options=options)

    try:
        # Crear una lista para almacenar los datos
        datos = []

        with open(csv_path, mode='r', encoding='utf-8') as archivo_csv:
            lector_csv = csv.reader(archivo_csv)
            next(lector_csv)  # Saltar la primera fila (encabezado)

            for indice, fila in enumerate(lector_csv, start=1):
                enlace = fila[1]  # La columna 1 contiene los enlaces
                titulo = fila[0]  # La columna 0 contiene los títulos

                # Verificar si el contenido de la página contiene palabras clave
                if buscar_palabras_clave_en_pagina(enlace, palabras_clave):
                    # Agregar los datos a la lista
                    datos.append([titulo, enlace])

                # Detener el procesamiento después de las primeras 5 filas
                if indice >= limite:
                    break

        # Imprimir los resultados
        for resultado in datos:
            print("Título:", resultado[0])
            print("Enlace:", resultado[1])
            print("-" * 50)

    finally:
        # Cerrar el navegador
        driver.quit()

if __name__ == "__main__":
    palabras_clave = ["Messi"]  # Personaliza las palabras clave según tus necesidades
    csv_path = 'resultados.csv'  # Ruta del archivo CSV
    limite_filas = 5  # Número de filas a procesar

    obtener_titulos_desde_csv(csv_path, limite=limite_filas)
